Importing necessary libraries

In [19]:
import numpy as np
import pandas as pd
import matplotlib as plt
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
import json

In [1]:
with open('NTUSD_Fin_word_v1.0.json') as json_file:
    data = json.load(json_file)

In [20]:
findict =[]
mktsnt =[]
for dictionary in data:
    findict.append(dictionary['token'])
    mktsnt.append(dictionary['market_sentiment'])

In [21]:
word_to_snt=dict(zip(findict, mktsnt))

In [22]:
ps_findict = PorterStemmer()
st_findict=[]
for word in findict:
    word=ps_findict.stem(word)
    st_findict.append(word)
#print(len(st_findict))

In [8]:
word_to_root = dict(zip(findict, st_findict))
#print(len(word_to_root))

In [9]:
df_word_to_root = pd.DataFrame.from_dict(word_to_root, orient = 'index',columns=['root'])
df_word_to_root.reset_index(inplace=True)
df_word_to_root.columns = ['word','root']
#print(len(df_word_to_root))
#df_word_to_root.head()

In [11]:
df_word_to_snt = pd.DataFrame.from_dict(word_to_snt, orient = 'index',columns=['snt'])
df_word_to_snt.reset_index(inplace=True)
df_word_to_snt.columns = ['word','snt']
#print(len(df_word_to_snt))
#df_word_to_snt.head()

In [13]:
df_concat = pd.concat([df_word_to_snt, df_word_to_root], axis=1)
df_concat.columns = ['word','snt','word1','root']
df_concat.drop(labels=['word1'],axis=1,inplace=True);
#df_concat.head()
#print(len(df_concat))
#df_concat.head()

In [15]:
df_concat_dropped = df_concat.drop_duplicates(subset ="root",keep = 'first') 
df_concat_dropped.reset_index(inplace=True)

In [16]:
final ={}
for row in df_concat_dropped.itertuples():
    tot=0
    count=0
    for row1 in df_concat.itertuples():
        if getattr(row, 'root')==getattr(row1, 'root'):
            tot=tot+getattr(row1, 'snt')
            count=count+1
    final[getattr(row, 'root')]=tot/count

In [18]:
with open('NTUSD_fin.json', 'w') as fp:
    json.dump(final, fp)